<a href="https://colab.research.google.com/github/Nahom32/Directed-Models/blob/main/medical_pgm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [3]:
import pandas as pd
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

## Task 1

In [4]:
random_variables = [
    "IsSummer", "HasFlu", "HasFoodPoisoning", "HasHayFever", "HasPneumonia",
    "HasRespiratoryProblems", "HasGastricProblems", "HasRash", "Coughs",
    "IsFatigued", "Vomits", "HasFever"
]
bayes_network = DiscreteBayesianNetwork([
    ("IsSummer", "HasFlu"),
    ("IsSummer", "HasFoodPoisoning"),
    ("IsSummer", "HasHayFever"),
    ("HasFlu", "HasRespiratoryProblems"),
    ("HasHayFever", "HasRespiratoryProblems"),
    ("HasPneumonia", "HasRespiratoryProblems"),
    ("HasFoodPoisoning", "HasGastricProblems"),
    ("HasHayFever", "HasRash"),
    ("HasRespiratoryProblems", "Coughs"),
    ("HasFlu", "IsFatigued"),
    ("HasPneumonia", "IsFatigued"),
    ("HasGastricProblems", "Vomits"),
    ("HasFlu", "HasFever"),
    ("HasPneumonia", "HasFever")
])



In [8]:
def load_dataset(file_path):
  with open(file_path, "r") as file:
    binary_data = []
    for line in file:
      binary_data.append(format(int(line.strip()), "012b"))
    data = [[int(b) for b in row] for row in binary_data]
    return pd.DataFrame(data, columns=random_variables)




In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
df = load_dataset("/content/drive/MyDrive/medical/dataset.dat")
print(df)

         IsSummer  HasFlu  HasFoodPoisoning  HasHayFever  HasPneumonia  \
0               0       0                 0            0             0   
1               0       0                 0            0             1   
2               0       0                 0            0             0   
3               0       0                 0            0             0   
4               0       0                 0            0             1   
...           ...     ...               ...          ...           ...   
3999995         0       0                 0            0             1   
3999996         1       0                 1            1             1   
3999997         0       0                 0            0             0   
3999998         0       0                 0            0             0   
3999999         0       0                 0            0             1   

         HasRespiratoryProblems  HasGastricProblems  HasRash  Coughs  \
0                             0        

In [12]:
bayes_network.fit(df, estimator=MaximumLikelihoodEstimator)
print("\nConditional Probability Table for HasFlu:")
print(bayes_network.get_cpds("HasFlu"))



Conditional Probability Table for HasFlu:
+-----------+---------------------+--------------------+
| IsSummer  | IsSummer(0)         | IsSummer(1)        |
+-----------+---------------------+--------------------+
| HasFlu(0) | 0.9083903219250574  | 0.8029588600104152 |
+-----------+---------------------+--------------------+
| HasFlu(1) | 0.09160967807494252 | 0.1970411399895848 |
+-----------+---------------------+--------------------+
